# PySpark Dataframes

1. Import the SparkSession class:

In [1]:
from pyspark.sql import SparkSession

To instiate a Spark session:

In [5]:
spark = SparkSession \
    .builder \
    .appName("First PySpark App") \
    .getOrCreate()

3. Take a look at the session object:


In [6]:
spark

4. Read the contents of a csv file into a Dataframe named 'accounts':


In [7]:
accounts = spark.read.option('header', 'true').csv('./data/accounts.csv')

5. Take a look at the Dataframe's schema:


In [8]:
accounts.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)



6. Read the contents of a parquet file into a variable:

In [11]:
transactions = spark.read.option("header", True).parquet('./data/transactions.parquet')

7. See how many rows are in the new Dataframe:

In [12]:
transactions.count()

1000000

In [16]:
transactions.show(5)

+------------------+------+----------+
|    account_number|amount|  datetime|
+------------------+------+----------+
|GSHP69974285041169| -9034|2022-09-08|
|LVTT31407534039456| -2952|2022-04-02|
|RJSO40371783238228|  3405|2022-05-02|
|UOIH17449189991636| -4049|2022-10-07|
|FBXP58394333953676| -4157|2022-06-22|
+------------------+------+----------+
only showing top 5 rows



8. Make a new Dataframe by grouping the transactions by account number and summing the groups. This will combine the transactions per account:

In [13]:
account_transactions = transactions.groupby('account_number').sum()

9. Combine the accounts with the summed transaction values:

In [14]:
with_sum = accounts.join(account_transactions, 'account_number', 'inner')

In [17]:
with_sum.show(10)

+------------------+---------+-----------+----------+-------+-----------+
|    account_number|      aba|        bic|    opened|balance|sum(amount)|
+------------------+---------+-----------+----------+-------+-----------+
|FBXK78425844480007|125056531|   TFJQGB5D|2014-05-07| 710027|     -99434|
|XJIU55438863095422|036174230|   KILTGB3K|2009-12-18| 339452|      77947|
|XBYT37304125118047|061333512|UVXHGB0SHJC|2013-12-21| 948840|      65101|
|UQSE17000937342665|028459374|JBPYGB7X207|2009-07-30| 928422|     118473|
|KWOU43650129218895|038163281|   PHLOGB1V|2016-06-14|1119190|     -35411|
|FNUS11547939884469|048504775|   KVXDGBQT|2001-06-23| 911119|     -61117|
|RWKR28406701224417|085114102|   INMGGB96|2013-10-13|1291803|      12973|
|NPLS84562938853238|125331256|   IHEAGB8O|2015-05-21| 933824|     -93579|
|FZHT89141815092168|018700127|NAIYGBBS2HS|2018-02-16|1926118|     -26762|
|KNLA77840498184902|093772264|   SFLUGBG7|2020-06-19|1291060|      46427|
+------------------+---------+--------

10. Get the current balance per account by summing the transaction sums with the initial account balance:

In [18]:
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))

11. Get accounts with negative current balances:

In [19]:
neg_balance = accounts.filter(accounts.new_balance < 0)

12. Read client data from a json file:

In [20]:
clients = spark.read.json('./data/clients.json')

13. Get the clients with a negative balance:


In [21]:
clients = clients.join(neg_balance, 'account_number', 'inner')

In [22]:
clients.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- address: string (nullable = true)
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- sum(amount): long (nullable = true)
 |-- new_balance: double (nullable = true)



14. Look at the top five clients with negative balances:

In [23]:
clients.select(['first_name','last_name','account_number', 'new_balance']).show(5)

+----------+---------+------------------+-----------+
|first_name|last_name|    account_number|new_balance|
+----------+---------+------------------+-----------+
|    Meagan| Sandoval|JMTP45763117901514|   -27573.0|
|  Michelle|   Knight|RBUE05237750254383|  -103459.0|
|      Paul|   Massey|RJMY57096756148587|   -58329.0|
|  Michelle|    Perez|ZYMB62177146259441|   -55431.0|
|     David|    Green|LRTH65732611614073|  -103831.0|
+----------+---------+------------------+-----------+
only showing top 5 rows



Name: Islam Mammadov \
Date: 06/24/2025